In [2]:
from datasets.sc_snet_code_dataset import ScSnetCodeDataset
%load_ext autoreload
%autoreload 2
from options.train_options import TrainOptions
from tqdm.notebook import tqdm
from datasets.ys_shapeset import ShapeNetZSets
import subprocess
import torch

In [4]:
opt = TrainOptions()
opt.phase = 'train'
opt.max_dataset_size=10000000

dataset = ScSnetCodeDataset()
dataset.initialize(opt)

In [58]:
ds = ShapeNetZSets(isTrain = True)

train_dl = torch.utils.data.DataLoader(
            ds,
            batch_size=24,
            shuffle=False,
            drop_last=True,
            num_workers=1)


In [59]:
for i, data in tqdm(enumerate(train_dl), total=len(train_dl)):
        x = 1

len(ds)


# for i in tqdm(range(len(ds))):
#     ds[i]

  0%|          | 0/1875 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
rc = subprocess.call("./launchers/train_rand_tf_snet_code.sh", shell=True)

------------ Options -------------
alpha: 0.75
batch_size: 24
bert_cfg: configs/bert2vq_shapeglot.yaml
cat: chair
checkpoints_dir: ./checkpoints
ckpt: None
continue_train: False
dataset_mode: owndataset
debug: 0
device: cuda
display_freq: 3000
gpu_ids: [0]
gpu_ids_str: 0
input_nc: 3
iou_thres: 0.0
isTrain: True
lambda_L1: 10.0
logs_dir: logs
lr: 0.0001
lr_decay_iters: 50
lr_policy: lambda
max_dataset_size: 10000000
model: rand_tf
nThreads: 9
n_less: 0
name: own-dataset-z-sets-to-sampled-set-rand_tf-owndataset-chair-LR1e-4-clean
ndf: 64
nepochs: 200
nepochs_decay: 50
ngf: 64
output_nc: 3
pix3d_mode: noBG
print_freq: 25
profiler: 0
ratio: 1.0
resnet2vq_ckpt: None
resnet_arch: resnet18
resnet_cfg: configs/resnet2vq_pix3d.yaml
resnet_ckpt: None
resnet_dset: None
resnet_model: None
resnet_norm: gn
save_epoch_freq: 3
save_latest_freq: 5000
seed: 111
serial_batches: False
snet_mode: noBG
tf_cfg: configs/rand_tf_snet_code.yaml
topk: 30
trunc_thres: 0.2
use_bin_sdf: 0
use_marginal: 0
vq_cat: ch

/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py3.8/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  0%|          | 0/1875 [00:00<?, ?it/s]
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
[*] autoregressively inferencing...:   0%|          | 0/512 [00:00<?, ?it/s]
Warning! Will not return all meshes
Warning! Will not return all meshes
Warning! Will not return all meshes
[*] autoregressively inferencing...:   0%|          | 0/512 [00:00<?, ?it/s]
Warning! Will not return all meshes
Warning! Will not return all meshes
Warning! Will not return all meshes
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE
torch.Size([12288]) torch.Size([512, 24, 512]) SHAPEEEE


Traceback (most recent call last):
  File "train.py", line 137, in <module>
    train_one_epoch()
  File "train.py", line 82, in train_one_epoch
    for i, data in tqdm(enumerate(train_dl), total=len(train_dl)):
  File "/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py3.8/lib/python3.8/site-packages/tqdm/notebook.py", line 254, in __iter__
    for obj in it:
  File "/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py3.8/lib/python3.8/site-packages/tqdm/std.py", line 1178, in __iter__
    for obj in iterable:
  File "/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py3.8/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 530, in __next__
    data = self._next_data()
  File "/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py3.8/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1224, in _next_data
    return self._process_data(data)
  File "/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py

In [11]:
import os
import time
import inspect

from termcolor import colored, cprint
from tqdm.notebook import tqdm

# profiler
from torch import profiler

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

from options.train_options import TrainOptions
from datasets.dataloader import CreateDataLoader, get_data_generator
from models.base_model import create_model

from utils.visualizer import Visualizer
from utils import util

opt = TrainOptions()
opt.phase = 'train'

seed =111
util.seed_everything(seed)

opt.dataset_mode='owndataset'
train_dl, test_dl = CreateDataLoader(opt)
train_ds, test_ds = train_dl.dataset, test_dl.dataset

test_dg = get_data_generator(test_dl)

dataset_size = len(train_ds)
if opt.dataset_mode == 'shapenet_lang':
    cprint('[*] # training text snippets = %d' % len(train_ds), 'yellow')
    cprint('[*] # testing text snippets = %d' % len(test_ds), 'yellow')
else:
    cprint('[*] # training images = %d' % len(train_ds), 'yellow')
    cprint('[*] # testing images = %d' % len(test_ds), 'yellow')

# main loop
model = create_model(opt)
cprint(f'[*] "{opt.model}" initialized.', 'cyan')

visualizer = Visualizer(opt)

# save model and dataset files
expr_dir = '%s/%s' % (opt.logs_dir, opt.name)
model_f = inspect.getfile(model.__class__)
dset_f = inspect.getfile(train_ds.__class__)
cprint(f'[*] saving model and dataset files: {model_f}, {dset_f}', 'blue')
modelf_out = os.path.join(expr_dir, os.path.basename(model_f))
dsetf_out = os.path.join(expr_dir, os.path.basename(dset_f))
os.system(f'cp {model_f} {modelf_out}')
os.system(f'cp {dset_f} {dsetf_out}')

if opt.vq_cfg is not None:
    vq_cfg = opt.vq_cfg
    cfg_out = os.path.join(expr_dir, os.path.basename(vq_cfg))
    os.system(f'cp {vq_cfg} {cfg_out}')
    
if opt.tf_cfg is not None:
    tf_cfg = opt.tf_cfg
    cfg_out = os.path.join(expr_dir, os.path.basename(tf_cfg))
    os.system(f'cp {tf_cfg} {cfg_out}')


# use profiler or not
if opt.profiler == '1':
    cprint("[*] Using pytorch's profiler...", 'blue')
    tensorboard_trace_handler = profiler.tensorboard_trace_handler(opt.tb_dir)
    schedule_args = {'wait': 2, 'warmup': 2, 'active': 6, 'repeat': 1}
    schedule = profiler.schedule(**schedule_args)
    activities = [profiler.ProfilerActivity.CPU, profiler.ProfilerActivity.CUDA]

    

################## main training loops #####################
def train_one_epoch(pt_profiler=None):
    global total_steps
    
    epoch_iter = 0
    for i, data in tqdm(enumerate(train_dl), total=len(train_dl)):
        iter_start_time = time.time()
        visualizer.reset()
        total_steps += opt.batch_size
        epoch_iter += opt.batch_size

        model.set_input(data)

        model.optimize_parameters(total_steps)

        nBatches_has_trained = total_steps // opt.batch_size

        # if total_steps % opt.print_freq == 0:
        if nBatches_has_trained % opt.print_freq == 0:
            errors = model.get_current_errors()

            t = (time.time() - iter_start_time) / opt.batch_size
            visualizer.print_current_errors(epoch, epoch_iter, total_steps, errors, t)

        if (nBatches_has_trained % opt.display_freq == 0) or i == 0:
            # eval
            model.inference(data)
            visualizer.display_current_results(model.get_current_visuals(), total_steps, phase='train')

            # model.set_input(next(test_dg))
            test_data = next(test_dg)
            model.inference(test_data)
            visualizer.display_current_results(model.get_current_visuals(), total_steps, phase='test')

        if total_steps % opt.save_latest_freq == 0:
            cprint('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps), 'blue')
            latest_name = f'epoch-latest'
            model.save(latest_name)
        
        if pt_profiler is not None:
            pt_profiler.step()


cprint('[*] Start training. name: %s' % opt.name, 'blue')
total_steps = 0
for epoch in range(opt.nepochs + opt.nepochs_decay):
    epoch_start_time = time.time()
    # epoch_iter = 0

    # profile
    if opt.profiler == '1':
         with profiler.profile(
            schedule=schedule,
            activities=activities,
            on_trace_ready=tensorboard_trace_handler,
            record_shapes=True,
            with_stack=True,
        ) as pt_profiler:
            train_one_epoch(pt_profiler)
    else:
        train_one_epoch()

    if epoch % opt.save_epoch_freq == 0:
        cprint('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps), 'blue')
        latest_name = f'epoch-latest'
        model.save(latest_name)
        cur_name = f'epoch-{epoch}'
        model.save(cur_name)

    # eval every 3 epoch
    if epoch % opt.save_epoch_freq == 0:
        metrics = model.eval_metrics(test_dl)
        visualizer.print_current_metrics(epoch, metrics, phase='test')
        print(metrics)

    cprint(f'[*] End of epoch %d / %d \t Time Taken: %d sec \n%s' %
        (
            epoch, opt.nepochs + opt.nepochs_decay,
            time.time() - epoch_start_time,
            os.path.abspath( os.path.join(opt.logs_dir, opt.name) )
        ), 'blue', attrs=['bold']
        )
    model.update_learning_rate()



[*] Dataset has been created: OwnDataset


AttributeError: 'TrainOptions' object has no attribute 'batch_size'